In [1]:
#Gerekli kütüphanelerin import edilmesi.Gereksiz kütüphaneler de bulunuyor olabilir. Silebilirsiniz.

import json
import os
from distutils.dir_util import copy_tree
import shutil
import pandas as pd

import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.applications.nasnet import NASNetLarge
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [28]:
#NASNetLarge CNN mimarisi girdi olarak 331x331 veri almaktadır. Batch size donanımınız yeterli ise arttırılabilir.

input_size = 331
batch_size_num = 2

dataset_path = "dataset/CDv2_Ornek"
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')

print(train_path)
print(val_path)

dataset/CDv2\train
dataset/CDv2\val


In [29]:
#ImageDataGenerator ile veriler dosyadan çekilirken gerçekleştirilecek augmentation yöntemleri ve parametreleri belirleniyor.
#Bu işlem sayesinde hem veriler eğitime hazır hale geliyor hem de overfitting'den kaçınılıyor.
#Validation verileine bu augmentation işlemleri uygulanmıyor.

train_datagen = ImageDataGenerator(
    rescale = 1./255,    
    rotation_range = 45,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (input_size, input_size),
    color_mode = "rgb",
    class_mode = "binary",
    batch_size = batch_size_num,
    shuffle = True
)

val_datagen = ImageDataGenerator(
    rescale = 1./255 
)

val_generator = val_datagen.flow_from_directory(
    directory = val_path,
    target_size = (input_size, input_size),
    color_mode = "rgb",
    class_mode = "binary",
    batch_size = batch_size_num,
    shuffle = True
)

Found 37968 images belonging to 2 classes.
Found 5473 images belonging to 2 classes.


In [30]:
#Bu hücrede model hazırlanmaktadır. Öncelikle NASNetLarge modeli import edilmekte. Bu modelin sınıflandırıcı katmanı devre dışı
#bırakılmakta ve katmanlar eğitilebilir olarak ayarlanmaktadır. Ardından sınıflandırıcı olarak bir dense katmanı eklenir.
#Modele uygun optimizer parametreleri de ayarlandıktan sonra model eğitime hazır hale gelmektedir.

from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

nasnetModel = NASNetLarge(include_top=False, weights='imagenet', input_shape=(331,331,3))
nasnetModel.trainable = True
model = Sequential()
model.add(nasnetModel)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 11, 11, 4032)      84916818  
                                                                 
 flatten_4 (Flatten)         (None, 487872)            0         
                                                                 
 dropout_4 (Dropout)         (None, 487872)            0         
                                                                 
 dense_4 (Dense)             (None, 1)                 487873    
                                                                 
Total params: 85,404,691
Trainable params: 85,208,023
Non-trainable params: 196,668
_________________________________________________________________


In [31]:
#EarlyStopping ve ModelCheckpoint callback fonksiyonlarının hazırlanması.

checkpoint_filepath = 'tmp_checkpoint'

custom_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=5,
        verbose=0, 
        mode='auto'
    ),
    ModelCheckpoint(
        filepath = os.path.join(checkpoint_filepath, 'best_model_NAS_CDv2'),
        monitor = 'val_loss',
        mode = 'min',
        verbose = 1,
        save_best_only = True
    )
]

In [25]:
#GPU yetersiz olduğu durumlarda tam güçle çalışmadan eğitimi sürdürmesini sağladı. 
#Fakat veri kümesi küçük kullanıldığında veya sizin donanımınız yeterli ise bu hücreye gerek yok.

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

D:\Programlar\anaconda3\lib\site-packages\tensorflow\python\client\session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [32]:
#Model eğitiminin başlatılması.

EPOCHS = 20
history = model.fit(
    train_generator,  
    epochs = EPOCHS, 
    validation_data = val_generator, 
    verbose = 1,
    callbacks = custom_callbacks
)

print(history.history)

Epoch 1/20
37968/37968 [==============================] - ETA: 0s - loss: 0.5433 - accuracy: 0.7009
Epoch 1: val_loss improved from inf to 0.48018, saving model to tmp_checkpoint\best_model_NAS_CDv2


INFO:tensorflow:Assets written to: tmp_checkpoint\best_model_NAS_CDv2\assets


INFO:tensorflow:Assets written to: tmp_checkpoint\best_model_NAS_CDv2\assets


37968/37968 [==============================] - 47513s 1s/step - loss: 0.5433 - accuracy: 0.7009 - val_loss: 0.4802 - val_accuracy: 0.8061
Epoch 2/20
37968/37968 [==============================] - ETA: 0s - loss: 0.2491 - accuracy: 0.8964
Epoch 2: val_loss did not improve from 0.48018
37968/37968 [==============================] - 28415s 748ms/step - loss: 0.2491 - accuracy: 0.8964 - val_loss: 1.3922 - val_accuracy: 0.9088
Epoch 3/20
37968/37968 [==============================] - ETA: 0s - loss: 0.1648 - accuracy: 0.9362
Epoch 3: val_loss did not improve from 0.48018
37968/37968 [==============================] - 27115s 714ms/step - loss: 0.1648 - accuracy: 0.9362 - val_loss: 1.6073 - val_accuracy: 0.9457
Epoch 4/20
37968/37968 [==============================] - ETA: 0s - loss: 0.1286 - accuracy: 0.9515
Epoch 4: val_loss did not improve from 0.48018
37968/37968 [==============================] - 26891s 708ms/step - loss: 0.1286 - accuracy: 0.9515 - val_loss: 3.0201 - val_accuracy: 0.938

In [33]:
#Modelin eğitim geçmişinin dosyaya kaydedilmesi.
import numpy as np
np.save('histories/my_history_CDv2.npy',history.history)

In [34]:
#Modelin kaydedilmesi.
model.save(os.path.join(checkpoint_filepath, 'model_CDv2.h5'))